<a href="https://colab.research.google.com/github/amindell11/WildfireSpreadPrediction/blob/main/notebooks/Test_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up Git/SSH

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [4]:
# --- Mount Drive if you haven't ---
from google.colab import drive
drive.mount('/content/drive')

import os, subprocess, pathlib, re

# 1️⃣  Path to the private key you generated earlier
key_path = pathlib.Path("/content/drive/MyDrive/.ssh/id_ed25519_colab")
assert key_path.exists(), "❌  SSH key not found on Drive. Generate it first."

# 2️⃣  Start ssh‑agent and capture its env vars
agent_output = subprocess.check_output(["ssh-agent", "-s"], text=True)
sock = re.search(r"SSH_AUTH_SOCK=([^;]+);", agent_output).group(1)
pid  = re.search(r"SSH_AGENT_PID=([0-9]+);", agent_output).group(1)
os.environ["SSH_AUTH_SOCK"] = sock
os.environ["SSH_AGENT_PID"] = pid

# 3️⃣  Add the key to the agent
subprocess.run(["ssh-add", str(key_path)], check=True)

# 4️⃣  Ensure ~/.ssh exists and pre‑populate known_hosts for GitHub
ssh_dir = pathlib.Path.home() / ".ssh"
ssh_dir.mkdir(exist_ok=True)            # ← replaces `mkdir -p ~/.ssh`
known_hosts = ssh_dir / "known_hosts"
subprocess.run(
    ["ssh-keyscan", "-t", "rsa", "github.com"],
    check=True, text=True,
    stdout=open(known_hosts, "a")
)

print("✅ SSH agent ready — you can git push over SSH from this runtime.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SSH agent ready — you can git push over SSH from this runtime.


In [17]:
username = "amindell11"
repo = "WildfireSpreadPrediction"

remote_url = f"git@github.com:{username}/{repo}.git"

%cd '/content/drive/MyDrive/repos/WildfireSpreadPrediction'
!git remote set-url origin {remote_url}
!git config --global user.email "amindell11@gmail.com"
!git config --global user.name "Arye Mindell"

/content/drive/MyDrive/repos/WildfireSpreadPrediction


In [6]:
!ssh -T git@github.com

Hi amindell11! You've successfully authenticated, but GitHub does not provide shell access.


In [18]:
!git add -u                 # or  !git add .
!git status -s
commit_msg = input("Commit message: ").strip()
if not commit_msg:
    commit_msg = "colab update"
!git commit -m "{commit_msg}"
!git pull --rebase origin main
!git push origin main

M  .gitignore
MM notebooks/Test_Book.ipynb
D  wandb/latest-run
D  wandb/run-20250420_205215-hiheexvi/files/requirements.txt
D  wandb/run-20250420_205215-hiheexvi/files/wandb-metadata.json
D  wandb/run-20250420_205215-hiheexvi/run-hiheexvi.wandb
Commit message: update gitignore (remove wandb run files) also git committing should finally work
[main afbf9f3] update gitignore (remove wandb run files) also git committing should finally work
 6 files changed, 11 insertions(+), 1125 deletions(-)
 rewrite notebooks/Test_Book.ipynb (98%)
 delete mode 120000 wandb/latest-run
 delete mode 100644 wandb/run-20250420_205215-hiheexvi/files/requirements.txt
 delete mode 100644 wandb/run-20250420_205215-hiheexvi/files/wandb-metadata.json
 delete mode 100644 wandb/run-20250420_205215-hiheexvi/run-hiheexvi.wandb
error: cannot pull with rebase: You have unstaged changes.
error: please commit or stash them.
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 th

# Install Dataset and Program modules

In [20]:
!pip install . --quiet

  Preparing metadata (setup.py) ... done


In [21]:
#copy kaggle auth key from drive to local
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [24]:
#download dataset to drive folder (Not necessary if dataset already downloaded)
!kaggle datasets download -d fantineh/next-day-wildfire-spread -p /content/drive/MyDrive/Kaggle/
!unzip /content/drive/MyDrive/Kaggle/next-day-wildfire-spread.zip -d /content/drive/MyDrive/Kaggle/next-day-wildfire-spread


Dataset URL: https://www.kaggle.com/datasets/fantineh/next-day-wildfire-spread
License(s): Attribution 4.0 International (CC BY 4.0)
next-day-wildfire-spread.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/drive/MyDrive/Kaggle/next-day-wildfire-spread.zip
replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_00.tfrecord  y
y
y
y
y

replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_01.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_eval_01.tfrecord  y
y
y
y
y
y

replace /content/drive/MyDrive/Kaggle/next-day-wildfire-spread/next_day_wildfire_spread_test_00.tfrecord? [y]es, [n]o, [A]ll, [N]one, [r]ename:   infl

In [25]:
data_loc = '/content/drive/MyDrive/Kaggle/next-day-wildfire-spread'
file_pattern = data_loc + '/next_day_wildfire_spread_train*'
file_pattern_test = data_loc + '/next_day_wildfire_spread_test*'
file_pattern_evaluate = data_loc + '/next_day_wildfire_spread_eval*'

In [26]:
import wildfire.autoencoder as auto_enc
import wildfire.utils as utils

config={
    "learning_rate": 0.0005,
    "architecture": "Convolutional Autoencoder",
    "dataset": "NDWS",
    "optimizer": "adam",
    "epochs": 100,
    "batch_size": 128,
    "dropout_rate": 0.10
}

kwargs = auto_enc.get_dataset_args(config.batchsize)

dataset = utils.get_dataset(file_pattern, **kwargs)
dataset_test = utils.get_dataset(file_pattern_test, **kwargs)
dataset_eval = utils.get_dataset(file_pattern_evaluate, batch_size=100, **kwargs)

model = auto_enc.build_autoencoder(input_shape=(64, 64, 12), dropout_rate=config.dropout_rate)

auto_enc.train_and_evaluate_model(model, dataset, dataset_test, dataset_eval, config)


ModuleNotFoundError: No module named 'wandb.keras'